In [ ]:
# import the required library
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By

# instantiate a Chrome options object
options = webdriver.ChromeOptions()

# set the options to use Chrome in headless mode
options.add_argument("--headless=new")

# initialize an instance of the chrome driver (browser)
driver = webdriver.Chrome(options=options)

driver.get("https://news.daum.net/employ")

# # extract all the product containers
# products = driver.find_elements(
#     By.CSS_SELECTOR,
#     ".box_news_basic .item_newsbasic"
# )

product_data = []

# for p in products:
#     title = p.find_element(By.CSS_SELECTOR, ".tit_txt").text
#     desc = p.find_element(By.CSS_SELECTOR, ".desc_txt").text
       
#     product_data.append({
#         "Title": title,
#         "Desc": desc,
#     })


section = driver.find_element(
    By.XPATH,
    "//div[contains(@class,'box_news_headline2')]//span[text()='취업・고용 주요뉴스']/ancestor::div[contains(@class,'box_comp')]"
)

second_products = section.find_elements(
    By.CSS_SELECTOR,
    ".list_newsheadline2 .item_newsheadline2"
)

for p in second_products:
    desc = p.find_element(By.CSS_SELECTOR, ".cont_thumb .tit_txt").text
       
    product_data.append({
        "Desc": desc,
    })

# release the resources allocated by Selenium and shut down the browser
driver.quit()

file_name = "daum_news.csv"

with open(file_name, mode="w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["Desc"]
    )
    writer.writeheader()
    writer.writerows(product_data)

print(f"{file_name} 저장 완료")

[<selenium.webdriver.remote.webelement.WebElement (session="3d8d6998598e466a912469b87a50e968", element="f.160601346628DB5943DDC16CB21821AC.d.C8CF4A9D002BF662DB1BC93212732A14.e.1075")>, <selenium.webdriver.remote.webelement.WebElement (session="3d8d6998598e466a912469b87a50e968", element="f.160601346628DB5943DDC16CB21821AC.d.C8CF4A9D002BF662DB1BC93212732A14.e.1089")>, <selenium.webdriver.remote.webelement.WebElement (session="3d8d6998598e466a912469b87a50e968", element="f.160601346628DB5943DDC16CB21821AC.d.C8CF4A9D002BF662DB1BC93212732A14.e.1102")>, <selenium.webdriver.remote.webelement.WebElement (session="3d8d6998598e466a912469b87a50e968", element="f.160601346628DB5943DDC16CB21821AC.d.C8CF4A9D002BF662DB1BC93212732A14.e.1115")>, <selenium.webdriver.remote.webelement.WebElement (session="3d8d6998598e466a912469b87a50e968", element="f.160601346628DB5943DDC16CB21821AC.d.C8CF4A9D002BF662DB1BC93212732A14.e.1128")>, <selenium.webdriver.remote.webelement.WebElement (session="3d8d6998598e466a9124

In [ ]:
import pandas as pd

df = pd.read_csv("daum_news.csv", encoding="utf-8-sig")

In [30]:
df.head()
df.columns


Index(['Desc'], dtype='object')

In [33]:
from konlpy.tag import Okt

okt = Okt()

tokens = []

for text in df["Desc"]:
    morphs = okt.morphs(text, stem=True)
    tokens.extend(morphs)

# 불용어 사전 (커스텀)
stopwords = {
    "리포트", "이코노미", "이코노미플러스", "포럼",
    "발간", "확대", "추진", "제공", "활용",
    "관련", "이번", "지난해", "이유", "세", "명",
    "전환", "별도", "시대",
}

tokens = [t for t in tokens if t not in stopwords and len(t) > 1]

for text in df["Desc"]:
    nouns = okt.nouns(text)
    for n in nouns:
        if n not in stopwords and len(n) > 1:
            tokens.append(n)


In [35]:
from collections import Counter
counter = Counter(tokens)


In [36]:
counter.most_common(20)


[('채용', 18),
 ('취업', 12),
 ('일자리', 8),
 ('청년', 8),
 ('근로자', 6),
 ('임금', 6),
 ('보험', 6),
 ('그리팅', 4),
 ('전략', 4),
 ('담당자', 4),
 ('핵심', 4),
 ('계절', 4),
 ('체불', 4),
 ('보증', 4),
 ('의무', 4),
 ('AI', 3),
 ('쉬다', 3),
 ('2026', 2),
 ('바다', 2),
 ('스타', 2)]

In [37]:
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

from wordcloud import WordCloud, STOPWORDS

In [ ]:
alice_mask = np.array(Image.open("alice_mask.png"))

In [ ]:
wc = WordCloud(background_color="white", max_words=2000, mask=alice_mask,
               stopwords=stopwords, contour_width=3, contour_color='steelblue')

# generate word cloud
wc.generate(tokens)

# store to file
wc.to_file("alice.png"))

# show
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.figure()
plt.imshow(alice_mask, cmap=plt.cm.gray, interpolation='bilinear')
plt.axis("off")
plt.show()